# Host a Forest Data Partnership trained model

Use this notebook to host [Forest Data Partnership](https://www.forestdatapartnership.org/) models on Vertex AI for predictions in Earth Engine.  The models are stored in TensorFlow saved model format in the [Forest Data Partnership GitHub repository](https://github.com/googlestaging/forest-data-partnership/tree/main).

**Note:** This guide uses paid components of Google Cloud Platform including Cloud Storage, Vertex AI and Earth Engine (commercial).  A Cloud project with billing enabled is needed for this workflow.

In [ ]:
import google
from google.colab import auth

In [ ]:
PROJECT = 'your-project'
BUCKET = 'your-writeable-bucket'

REGION = 'us-central1'
CONTAINER_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-13:latest'
MODEL_NAME = 'palm_model_20240312'
ZIP_NAME = f'{MODEL_NAME}_hostable.zip'
MODEL_PATH = f'{MODEL_NAME}_hostable'
ARTIFACT_URI = f'gs://{BUCKET}/{MODEL_PATH}'
GH_PATH = f'https://raw.githubusercontent.com/google/forest-data-partnership/main/models/{ZIP_NAME}'
ENDPOINT_NAME = MODEL_NAME + '_endpoint'

In [ ]:
auth.authenticate_user()
credentials, project = google.auth.default()

## Download a trained model from the GitHub repo

In [ ]:
!curl -o {ZIP_NAME} {GH_PATH}

In [ ]:
!unzip {ZIP_NAME}

In [ ]:
!gsutil cp -r {MODEL_PATH} gs://{BUCKET}

### [Upload the model](https://cloud.google.com/sdk/gcloud/reference/ai/models/upload)
Add an entry to the model registry that points to the location of the saved model and a container image needed to run the model.

In [ ]:
!gcloud ai models upload \
  --project={PROJECT} \
  --region={REGION} \
  --artifact-uri={ARTIFACT_URI} \
  --container-image-uri={CONTAINER_IMAGE} \
  --description={MODEL_NAME} \
  --display-name={MODEL_NAME} \
  --model-id={MODEL_NAME}

### [Create an endpoint](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/create)

Create an endpoint from which to serve the model.

In [ ]:
!gcloud ai endpoints create \
  --display-name={ENDPOINT_NAME} \
  --region={REGION} \
  --project={PROJECT}

### [Deploy the model to the endpoint](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/deploy-model)

First, look up the endpoint ID, then deploy the model.

In [ ]:
ENDPOINT_ID = !gcloud ai endpoints list \
  --project={PROJECT} \
  --region={REGION} \
  --filter=displayName:{ENDPOINT_NAME} \
  --format="value(ENDPOINT_ID.scope())"
ENDPOINT_ID = ENDPOINT_ID[-1]

In [ ]:
print(ENDPOINT_ID)

In [ ]:
!gcloud ai endpoints deploy-model {ENDPOINT_ID} \
  --project={PROJECT} \
  --region={REGION} \
  --model={MODEL_NAME} \
  --display-name={MODEL_NAME}

In [ ]:
print('Prediction link:')
print(f'https://code.earthengine.google.com/7ddb7bba1beecf0a9942fec36371b248#project={PROJECT};endpoint={ENDPOINT_ID}foo;')